<h1 style="display: flex; align-items: center; justify-content: center; font-weight: bold; font-size: 2em;">
  <img src="/Users/jasonjoelpinto/Documents/GitHub/python-datascience-projects/016. sentiment_analysis_NLP_project/dataset/icons8-twitter-pastel/icons8-twitter-256.png" alt="Your Image" style="height: 2.5em; margin-right: 15px;">
  Twitter Post Classification
</h1>


## 👋🏼 **Introduction**

<p style="text-align: justify;">I am a data scientist. And my company wants me to build and NLP pipeline where it can categorize the tweets into suicidical or non suicidical tweets. The problem that currently most of them is facing is there are alot of times when people tweet about what they are feeling or when they are having suicidical taughts. But most of the times no one is there to help them. Hence the idea to to build a model where it can detect if it's a tweet related to suicide and give any sorts of help that the person needs.</p>

## 🧿 **Objective**
 <p style="text-align: justify;">The objective of this project is that I will have to build a module which will try to classify the category of twitter reviews. For this my company has given me a dataset which is in csv format. It has three columns, id, tweet and the label. So I will be using these data in the csv file and building an NLP model ???????. Firstly I will be going through the full csv dataset and trying to remove the unwanted data if unsupported extensions. And then using these data I will be building a model. I will also be doing several experiments in order to improve the performance of the model. I will check if our dataset is balanced and if there is a requirement to increase the dataset so that our model can learn better from the data. Then I will choose the best-performing model to build my final model with the best-suited parameters.
 </p>

**About the data**  
This dataset contains 12 columns. This data set is from Kaggle. You can [**click here**](https://www.kaggle.com/datasets/mayurdalvi/twitter-sentiments-analysis-nlp) to view the dataset on kaggle.

## **Table of Content**

##### ⬇ [**Importing Libraries**](#library)
##### 📊 [**Importing Dataset**](#dataset)
##### 🗺️ [**Exploring the dataset**](#exploration)
##### 💭 [**My taughts on the dataset**](#thoughts)
##### ⚙️ [**Pre-processing the data**](#processing)
##### 🔧 [**Feature Engineering**](#feature)
##### 🗂️ [**Selecting the best model for our dataset**](#selection)
##### ✅ [**Assessing which gives us the best results**](#assess)
##### 🎛️ [**Hyper-Parameter Tuning**](#tuning)
##### 🏁 [**Final Model**](#Final)
##### 💡 [**Conclusion**](#conclusion)


## ***Let's begin....***

<a id='library'></a>
## ⬇ <span style="color: #20479b; font-weight: bold;">Importing Libraries</span>

I start by importing the necessary libraries for my Exploratory Data Analysis tasks in Python.

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import seaborn as sns
import os
import warnings
import string
import emoji

from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.model_selection
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.metrics import geometric_mean_score

<a id='dataset'></a>  

## 📊 <span style="color: #20479b; font-weight: bold;">Importing Dataset</span>

In [ ]:
dataset_dir = '/Users/jasonjoelpinto/Documents/GitHub/python-datascience-projects/016. sentiment_analysis_NLP_project/dataset/Twitter Sentiments.csv'
dataset = pd.read_csv(dataset_dir, nrows=10000)
print(display(dataset.sample(5)))

<a id='exploration'></a>
## 🗺️ <span style="color: #20479b; font-weight: bold;">Exploring the dataset</span>

In [ ]:
dataset.sample(5)

I can see that 
1. ID column is not necessary for us, hence we can delete it.
2. tweets have alot of unnessary special characters, which we have to clean.

In [ ]:
dataset.shape

There are 31962 rows of data which is good so that the model can learn better from the dataset.

Here as we can see there are three columns i.e. id, label and tweet.
As per the data authors, 1 here stands for rasist/sexist tweet and 0 stands for non racist comment.

In [ ]:
dataset.dtypes

There are no null entries in our dataset.

There are no unnessary entries too. Which is a good thing for us.

Here we can see that our dataset is **heavily imbalanced**. We need to balance the dataset so that we can have good model trained.

<a id='thoughts'></a>
## 💭 <span style="color: #20479b; font-weight: bold;">My thoughts on the dataset</span>

As per my initial Data Exploration,
- I can see that the dataset big enough to build a good ML model.
- I need to removed the ID column as we do not need it. 
- I will be also restructuring the column and moving the labels to the end to ease of reading.
- The dataset consists of 31962 rows of data.
- There are usertags and special characters in the column tweet. We need to remove those.
- The dataset is heavily imbalanced. Hence we need to oversample the data.


Evaluation Metrics:
For this dataset I will be using, Accuracy, precision, F1 are the evaluation metrics.

Considering all these above points, I will be following the below steps to 

**⚙️ pre-process**
1. 🗑️ Delete the column ID.
2. 🫸🏻 Move the column label to the end.
3. 🔄 Convert the body to lower case
4. 🧹 Remove usertags.
5. 🧹 Remove special characters.
<!-- 4. ➗ Spliting our main datset into two parts, one part for training and another for testing dataframe
    - Doing it before testing or 
    - to avoid data Scalling and oversampling. -->

**🔧 feature Engineer**
<!-- 1. ⚖️ Scalling the numerical and categorical labels -->

<!-- 2. 📈 Oversampling the data for stroke = 1, Because of two main reasons.
    - Our focus is on stroke detection and data for people who have stroke is very less.
    - We do not have to worry about data size as well here, since it's not very huge. -->

<a id='processing' ></a>
## ⚙️ <span style="color: #20479b; font-weight: bold;">Pre-processing the data</span>


**1. 🗑️ Delete the column ID**

In [ ]:
dataset = dataset.drop('id', axis=1)
dataset.sample(5)

**2. 🫸🏻 Move the column label to the end**

In [ ]:
new_col_order = ['tweet', 'label']
dataset = dataset[new_col_order]
dataset.sample(5)

**3. 🔄 Convert the body to lower case**

In [ ]:
dataset['tweet_processed'] = dataset['tweet'].str.lower()
dataset.sample(5)

**4. #️⃣ Remove hashtags**

In [ ]:
dataset['tweet_processed'] = dataset['tweet_processed'].replace(r'#([^\s]+)', r'\1', regex=True)
dataset.head(5)

**5.  🧹 Remove usertags**

In [ ]:
dataset['tweet_processed'] = dataset['tweet_processed'].replace(r'@[^\s]+', '', regex=True)
dataset.head(5)

**6.🗑️ Remove Punctuation**

In [ ]:
def remove_punctuation(tweet):
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    return tweet

dataset['tweet_processed'] = dataset['tweet_processed'].apply(remove_punctuation)
dataset.sample(5)

**7. 🔢 Clear all numbers**

In [ ]:
dataset['tweet_processed'] = dataset['tweet_processed'].replace(r'\d+', '', regex=True)
dataset.sample(5)

**8. 🧹 Remove special characters**

In [ ]:
dataset['tweet_processed'] = dataset['tweet_processed'].str.replace("[^a-zA-Z#]", " ")
dataset.head()

**9. Remove unwanted giberish characters**

In [ ]:
def remove_gibberish_and_space(tweet):
    tweet = tweet.encode('ascii', 'ignore').decode('ascii')
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet

dataset['tweet_processed'] = dataset['tweet_processed'].apply(remove_gibberish_and_space)
dataset.head(10)

In [ ]:

# porter = nltk.SnowballStemmer('english')

# def preprocessor(input_text):
#     tokenized_text = nltk.word_tokenize(input_text, language='english')
#     stems = [porter.stem(t) for t in tokenized_text]
#     return ' '.join(stems)

# dtst['tweet_processed'] = dtst['tweet_processed'].apply(preprocessor)

# dtst.head(10)

**10. Creating a TD-IDF matrix for the tweets**

In [ ]:
# dtst['tweet_processed'] = dtst['tweet_processed'].fillna('')

In [ ]:
vectorizer = TfidfVectorizer(min_df=2)
dataset = dataset.join(pd.DataFrame(vectorizer.fit_transform(dataset['tweet_processed']).toarray()))
inverted_index = {id: term for term, id in vectorizer.vocabulary_.items()}
dataset.head(5)

There are unwanted characters in my tweets data. How can I remove those? Here is some examples

In [ ]:
# text = '#model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦'
# hi = text.encode('ascii', 'ignore').decode('ascii')
# hi

**11. ❎ Splitting the data**

In [ ]:
target = dataset['label'].values.astype(float)

features = dataset.drop(columns=['tweet', 'tweet_processed', 'label']).values.astype(float)

In [ ]:
train_features, val_test_features, train_target, val_test_target = sklearn.model_selection.train_test_split(  features , target  ,  test_size = 0.35, shuffle=True, stratify=target, random_state  =  40)
val_features,  test_features,   val_target,  test_target = sklearn.model_selection.train_test_split(  val_test_features , val_test_target  ,  test_size = 0.50,shuffle=True, stratify=val_test_target,random_state  =  40)

print(f'Train features shape     : {train_features.shape}')
print(f'Train target shape       : {train_target.shape}')
print(39*'=')
print(f'Validation features shape: {val_features.shape}')
print(f'Validation target shape  : {val_target.shape}')
print(39*'=')
print(f'Test features shape      : {test_features.shape}')
print(f'Test target shape        : {test_target.shape}')
print(39*'=')


<a id='feature' ></a>
## 🔧 <span style="color: #20479b; font-weight: bold;">Feature Engineering</span>


**1. 📈 Oversampling the data**

Applying **SMOTE** oversampling to the dataset

In [ ]:
# **12. Balancing the data using SMOTE**
# ```python
print(f'Original Training Target Data Distribution: {np.bincount(train_target.astype(int))}')

# Apply SMOTE to balance the training data
smote = SMOTE(random_state=0)
train_features, train_target = smote.fit_resample(train_features, train_target)

print(f'Resampled Training Target Data Distribution: {np.bincount(train_target.astype(int))}')

Applying **NGRAM** oversampling to the dataset

In [ ]:
# Function to extract n-grams
def generate_ngrams(text, n=2):
    words = text.split()
    ngrams = zip(*[words[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

dataset['bigrams'] = dataset['tweet_processed'].apply(lambda x: generate_ngrams(x, 2))
dataset['trigrams'] = dataset['tweet_processed'].apply(lambda x: generate_ngrams(x, 3))

# Convert n-grams into separate columns in the dataframe
bigrams_vectorizer = TfidfVectorizer(min_df=2)
trigrams_vectorizer = TfidfVectorizer(min_df=2)

bigrams_matrix = bigrams_vectorizer.fit_transform(dataset['bigrams'].apply(lambda x: " ".join(x)))
trigrams_matrix = trigrams_vectorizer.fit_transform(dataset['trigrams'].apply(lambda x: " ".join(x)))

# Convert sparse matrix to dataframe and concatenate with the original dataframe
bigrams_df = pd.DataFrame(bigrams_matrix.toarray(), columns=[f'bigram_{i}' for i in range(bigrams_matrix.shape[1])])
trigrams_df = pd.DataFrame(trigrams_matrix.toarray(), columns=[f'trigram_{i}' for i in range(trigrams_matrix.shape[1])])

dataset = pd.concat([dataset, bigrams_df, trigrams_df], axis=1)

SENTIMENT SCORE

In [ ]:
from textblob import TextBlob

# Function to calculate sentiment score
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

dataset['sentiment_score'] = dataset['tweet_processed'].apply(get_sentiment)

In [ ]:
logistic_model = LogisticRegression()
logistic_model.fit(train_features, train_target)

logistic_predictions = logistic_model.predict(val_features)

print(f'Logistic Regression Accuracy: {accuracy_score(val_target, logistic_predictions)}')
print(f'Logistic Regression F1 Score: {f1_score(val_target, logistic_predictions)}')

In [ ]:
random_forest_model = RandomForestClassifier()
random_forest_model.fit(train_features, train_target)

rf_predictions = random_forest_model.predict(val_features)

print(f'Random Forest Accuracy: {accuracy_score(val_target, rf_predictions)}')
print(f'Random Forest F1 Score: {f1_score(val_target, rf_predictions)}')

In [ ]:
gb_model = GradientBoostingClassifier()
gb_model.fit(train_features, train_target)

gb_predictions = gb_model.predict(val_features)

print(f'Gradient Boosting Accuracy: {accuracy_score(val_target, gb_predictions)}')
print(f'Gradient Boosting F1 Score: {f1_score(val_target, gb_predictions)}')

<a id='feature' ></a>
## 🔧 <span style="color: #20479b; font-weight: bold;">Feature Engineering</span>

In [ ]:
def product_score(y_true, y_pred):
    return geometric_mean_score(y_true, y_pred) * accuracy_score(y_true, y_pred) * f1_score(y_true, y_pred)

def apply_model(clf_class, **kwargs):
    clf = clf_class(**kwargs)
    print(clf_class)
    print(35 * '=')

    clf.fit(ft_train_df, tr_train_df)
    pred = clf.predict(ft_val_df)
    gmean = geometric_mean_score(tar_val_df, pred)
    acc = accuracy_score(tar_val_df, pred)
    f1 = f1_score(tar_val_df, pred)
    prod = product_score(tar_val_df, pred)
    print('Imbalanced: G-Mean = {:.3f} Accuracy = {:.3f} f1 = {:.3f} Product = {:.3f}'.format(gmean, acc, f1, prod))

    clf.fit(ft_train_df_smt, tr_train_df_smt)
    pred = clf.predict(ft_val_df)
    gmean = geometric_mean_score(tar_val_df, pred)
    acc = accuracy_score(tar_val_df, pred)
    f1 = f1_score(tar_val_df, pred)
    prod = product_score(tar_val_df, pred)
    print('SMOTE: G-Mean = {:.3f} Accuracy = {:.3f} f1 = {:.3f} Product = {:.3f}'.format(gmean, acc, f1, prod))

    clf.fit(ft_train_df_adc, tr_train_df_adc)
    pred = clf.predict(ft_val_df)
    gmean = geometric_mean_score(tar_val_df, pred)
    acc = accuracy_score(tar_val_df, pred)
    f1 = f1_score(tar_val_df, pred)
    prod = product_score(tar_val_df, pred)
    print('ADASYN: G-Mean = {:.3f} Accuracy = {:.3f} f1 = {:.3f} Product = {:.3f}'.format(gmean, acc, f1, prod))

    print('\n')

def compare_models():
    apply_model(GradientBoostingClassifier)
    apply_model(RandomForestClassifier)
    apply_model(LogisticRegression)

compare_models()

<a id='selection'></a>
## 🗂️ <span style="color: #20479b; font-weight: bold;">Experimental Section</span>

So I have 10 different configurations and choose the best out of these

<style>
  table {
    width: 100%;
    border-collapse: collapse;
    border: 1px solid #ddd;
    font-family: Arial, sans-serif;
  }
  th, td {
    padding: 12px;
    text-align: left;
    border-bottom: 1px solid #ddd;
  }
  th {
    background-color: #03bafc;
    color: #000;
  }
  tr:nth-child(even) {
    background-color: #EEEEEE;
  }
  tr:hover {
    background-color: #DBECFA; /* Light tint of blue for hover effect */
  }
  td:first-child {
    font-weight: bold;
  }
   tr.mdl_config8 {
    background-color: #95CD97; /* Dark light green */
  } 
  .mdl_config8 {
    background-color: #95CD97; /* Darker green for note */
    padding: 8px;
    font-style: italic;

  }
</style>

<table>
  <thead>
    <tr>
      <th>Model Config</th>
      <th>Layers</th>
      <th>Validation Accuracy</th>
      <th>Observations</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><b>mdl_config1</b></td>
      <td>Flatten(input_shape=(256,256,3))<br>Dense(4, activation='softmax')</td>
      <td>0.8375</td>
      <td>The accuracy was very low and it was fluctuating a lot. Hence, I will add one more dense hidden layer to the model.</td>
    </tr>
    <tr>
      <td><b>mdl_config2</b></td>
      <td>Flatten(input_shape=(256,256,3))<br>Dense(16, activation='relu')<br>Dense(4, activation='softmax')</td>
      <td>0.4312</td>
      <td>With the additional layer with 16 neurons, the accuracy dipped to 43%. I wanted to check if making the network more complex would help the model predict better.</td>
    </tr>
    <tr>
      <td><b>mdl_config3</b></td>
      <td>Flatten(input_shape=(256,256,3))<br>Dense(16, activation='relu')<br>tf.keras.layers.Dense(32, activation='relu')<br>Dense(4, activation='softmax')</td>
      <td>0.63125</td>
      <td>With the addition of another layer for 16 neurons, the accuracy increased by approximaterly 20%. Since these are images I wanted to check addition of convolutional and maxpooling layer will help, I will also increase the neurons for the dense layer</td>
    </tr>
    <tr>
      <td><b>mdl_config4</b></td>
      <td>Conv2D(16, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Flatten()<br>Dense(256, activation='relu')<br>Dense(4, activation='softmax')</td>
      <td>0.93125</td>
      <td>Due to the convlutional layers it was able to read the images better. It was able to extract various features from the dataset. The accuracy has not increased to 93. Now I want to check it adding one more layer will help increase the accuracy.</td>
    </tr>
    <tr>
      <td><b>mdl_config5</b></td>
      <td>Conv2D(16, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(32, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Flatten()<br>Dense(256, activation='relu')<br>Dense(4, activation='softmax')</td>
      <td>0.93750</td>
      <td>There was no significant increace in the accuracy after adding another convolutional layer. But it was still slightly better than the previous configuration. Just to experiment I want to add another layer of convolution with more neurons and see if it increases the performance.</td>
    </tr>
    <tr>
      <td><b>mdl_config6</b></td>
      <td>Conv2D(16, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(32, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(64, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Flatten()<br>Dense(256, activation='relu')<br>Dense(4, activation='softmax')</td>
      <td>0.99375</td>
      <td>This was a huge improvement for me. The accuracy went to 99. The accuracy is fluctuating a little bit though. I do not want the model to overfit the train dataset hence I will experiment adding Early Stopping and see how it impacts</td>
    </tr>
    <tr>
      <td><b>mdl_config7</b></td>
      <td>Conv2D(16, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(32, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(64, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Flatten()<br>Dense(256, activation='relu')<br>Dense(4, activation='softmax')<br>++EarlyStopping</td>
      <td>0.996324</td>
      <td>The accuracy has improved slightly and the accuracy curve is also much better. To avoid further overfit I will add Dropout layer see how it impacts</td>
    <tr class="mdl_config8">
      <td><b>👑 mdl_config8</b></td>
      <td>Conv2D(16, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(32, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(64, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Flatten()<br>Dense(256, activation='relu')<br>Dropout(0.5)<br>Dense(4, activation='softmax')<br>++EarlyStopping</td>
      <td>0.98750</td>
      <td class="observations">The accuracy has decreased slightly, this may be because it was overfitting the data earlier. Let me add batch normalization and check if it will improve the accuracy.
      <br><br><b>Note:</b> I have chosen this even if model 7 has better results because this model has both Early Stopping and Dropout, and even with that, it was able to score 0.98750 without overfitting the train data, which is great!!</td>
    </tr>
    <tr>
      <td><b>mdl_config9</b></td>
      <td>Conv2D(16, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(32, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(64, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Flatten()<br>Dense(256, activation='relu')<br>BatchNormalization()<br>Dropout(0.5)<br>Dense(4, activation='softmax')<br>++EarlyStopping</td>
      <td>0.98750</td>
      <td>The accuracy is still the same. Now I want to check if changing the optimizers to sgd from adam will help increase the acccuracy</td>
    </tr>
    <tr>
      <td><b>mdl_config10</b></td>
      <td>Conv2D(16, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(32, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(64, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Flatten()<br>Dense(256, activation='relu')<br>BatchNormalization()<br>Dropout(0.5)<br>Dense(4, activation='softmax')<br>++EarlyStopping</td>
      <td>0.98750</td>
      <td>With sgd the accuracy dropped to 72, and the accuracy fluctuates alot. So I will be going back to the adam optimizer. And I will try increasing the learning rate</td>
    </tr>
    <tr>
      <td><b>mdl_config11</b></td>
      <td>Conv2D(16, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(32, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Conv2D(64, (2,2), 1, activation='relu', input_shape=(256,256,3))<br>MaxPooling2D()<br>Flatten()<br>Dense(256, activation='relu')<br>BatchNormalization()<br>Dropout(0.5)<br>Dense(4, activation='softmax')<br>++EarlyStopping</td>
      <td>0.98750</td>
      <td>With increase in learning rate thethe accuracy fluctuates even more.</td>
    </tr>
  </tbody>
</table>


<a id='Final'></a>
## 🏁 <span style="color: #20479b; font-weight: bold;">Final Model</span>

In [ ]:
mdl_config8 = Sequential([
    tf.keras.layers.Conv2D(16, (2,2), 1, activation='relu', input_shape=(256,256,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, (2,2), 1, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (2,2), 1, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])

mdl_config8.compile('adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
early_stop = tf.keras.callbacks.EarlyStopping(monitor='validation_loss',mode='max',patience=4, restore_best_weights=True)
hist_config8 = mdl_config8.fit(train_btc_dt, epochs=20, validation_data=validation_btc_dt, callbacks=[early_stop])
print('')
print('=====================================================================================================')
print('')

mdl_config8.summary()

fig, (acc, lss) = plt.subplots(1, 2, figsize=(7.5, 3))

acc.plot(hist_config8.history['accuracy'], color='red', label='accuracy')
acc.plot(hist_config8.history['val_accuracy'], color='blue', label='val_accuracy')
acc.set_title('Accuracy', fontsize=20)
acc.legend(loc="upper left")

lss.plot(hist_config8.history['loss'], color='red', label='train_loss')
lss.plot(hist_config8.history['val_loss'], color='blue', label='validation_loss')
lss.set_title('Loss', fontsize=20)
lss.legend(loc="upper right")

plt.show()

<a id='Evaluation'></a>
## 💡 <span style="color: #20479b; font-weight: bold;"> Model Evaluation</span>

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

recl_lst = []
prcn_lst = []
acry_lst = []

imgs = []
tarpreds = []
tars = []

for btc in test_btc_dt.as_numpy_iterator():
    img, tar = btc
    tarpred = mdl_config8.predict(img)
    tarpred = tarpred.argmax(axis=1)
    recln = recall_score(tar, tarpred, average='macro')
    prcnn = precision_score(tar, tarpred, average='macro')
    acryn = accuracy_score(tar, tarpred)
    prcn_lst.append(prcnn)
    recl_lst.append(recln)
    acry_lst.append(acryn)

    # Append images, predictions, and ground truths for visualization
    imgs.extend(img)
    tarpreds.extend(tarpred)
    tars.extend(tar)

    # Break the loop if enough images are collected
    if len(imgs) >= 6:
        break

print(f'Precision : {sum(prcn_lst)/len(prcn_lst)}')
print(f'Recall    : {sum(recl_lst)/len(recl_lst)}')
print(f'Accuracy  : {sum(acry_lst)/len(acry_lst)}')

fig, am = plt.subplots(2, 3, figsize=(12, 8))
am = am.ravel()

for i,img in enumerate(imgs):
    # Plot image
    am[i].imshow(img)
    am[i].set_xticks([])
    am[i].set_yticks([])
    
    if tarpreds[i] == tar[i]:
        ttl = 'green'
    else:
        ttl = 'red'
    
    am[i].set_title(f'Predicted: {eq_class[tarpreds[i]]}\nActual: {eq_class[tars[i]]}', color=ttl)

    if i >=5:
        break


plt.tight_layout()
plt.show()

As we can see in the above plot, we were able to get very good scored and out of the 6 images we were able to predict most of it.

<a id='conclusion'></a>
## 💡 <span style="color: #20479b; font-weight: bold;">Conclusion</span>

<a target="_blank" href="https://icons8.com/icon/68193/twitter">Twitter</a> icon by <a target="_blank" href="https://icons8.com">Icons8</a>